# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 3 2023 9:15AM,258611,10,37788,"Digital Brands, LLC",Released
1,Mar 3 2023 9:15AM,258611,10,37789,"Digital Brands, LLC",Released
2,Mar 3 2023 9:15AM,258611,10,37790,"Digital Brands, LLC",Released
3,Mar 3 2023 9:09AM,258610,19,9226970,"Graystone, LLC",Released
4,Mar 3 2023 9:08AM,258609,19,9226974,"Graystone, LLC",Released
5,Mar 3 2023 9:01AM,258607,10,8896970,Eywa Pharma Inc.,Released
6,Mar 3 2023 9:01AM,258608,10,Enova-11452,Emerginnova,Released
7,Mar 3 2023 9:01AM,258608,10,Enova-11453,Emerginnova,Released
8,Mar 3 2023 9:01AM,258608,10,Enova-11454,Emerginnova,Released
9,Mar 3 2023 9:01AM,258608,10,Enova-11455,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,258607,Released,1
20,258608,Released,5
21,258609,Released,1
22,258610,Released,1
23,258611,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258607,NaN,1.0
258608,NaN,5.0
258609,NaN,1.0
258610,NaN,1.0
258611,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258523,4.0,1.0
258526,0.0,5.0
258528,0.0,5.0
258530,0.0,5.0
258533,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258523,4,1
258526,0,5
258528,0,5
258530,0,5
258533,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258523,4,1
1,258526,0,5
2,258528,0,5
3,258530,0,5
4,258533,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258523,4,1
1,258526,,5
2,258528,,5
3,258530,,5
4,258533,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 3 2023 9:15AM,258611,10,"Digital Brands, LLC"
3,Mar 3 2023 9:09AM,258610,19,"Graystone, LLC"
4,Mar 3 2023 9:08AM,258609,19,"Graystone, LLC"
5,Mar 3 2023 9:01AM,258607,10,Eywa Pharma Inc.
6,Mar 3 2023 9:01AM,258608,10,Emerginnova
11,Mar 3 2023 8:50AM,258605,10,"Senseonics, Incorporated"
12,Mar 3 2023 8:47AM,258604,10,Ivaoes Animal Health
13,Mar 3 2023 8:46AM,258603,10,"Nextsource Biotechnology, LLC"
15,Mar 3 2023 8:45AM,258602,10,Eye Pharma Inc
17,Mar 3 2023 8:32AM,258599,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 3 2023 9:15AM,258611,10,"Digital Brands, LLC",,3
1,Mar 3 2023 9:09AM,258610,19,"Graystone, LLC",,1
2,Mar 3 2023 9:08AM,258609,19,"Graystone, LLC",,1
3,Mar 3 2023 9:01AM,258607,10,Eywa Pharma Inc.,,1
4,Mar 3 2023 9:01AM,258608,10,Emerginnova,,5
5,Mar 3 2023 8:50AM,258605,10,"Senseonics, Incorporated",,1
6,Mar 3 2023 8:47AM,258604,10,Ivaoes Animal Health,,1
7,Mar 3 2023 8:46AM,258603,10,"Nextsource Biotechnology, LLC",,2
8,Mar 3 2023 8:45AM,258602,10,Eye Pharma Inc,,2
9,Mar 3 2023 8:32AM,258599,12,Hush Hush,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 9:15AM,258611,10,"Digital Brands, LLC",3,
1,Mar 3 2023 9:09AM,258610,19,"Graystone, LLC",1,
2,Mar 3 2023 9:08AM,258609,19,"Graystone, LLC",1,
3,Mar 3 2023 9:01AM,258607,10,Eywa Pharma Inc.,1,
4,Mar 3 2023 9:01AM,258608,10,Emerginnova,5,
5,Mar 3 2023 8:50AM,258605,10,"Senseonics, Incorporated",1,
6,Mar 3 2023 8:47AM,258604,10,Ivaoes Animal Health,1,
7,Mar 3 2023 8:46AM,258603,10,"Nextsource Biotechnology, LLC",2,
8,Mar 3 2023 8:45AM,258602,10,Eye Pharma Inc,2,
9,Mar 3 2023 8:32AM,258599,12,Hush Hush,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 9:15AM,258611,10,"Digital Brands, LLC",3,
1,Mar 3 2023 9:09AM,258610,19,"Graystone, LLC",1,
2,Mar 3 2023 9:08AM,258609,19,"Graystone, LLC",1,
3,Mar 3 2023 9:01AM,258607,10,Eywa Pharma Inc.,1,
4,Mar 3 2023 9:01AM,258608,10,Emerginnova,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 9:15AM,258611,10,"Digital Brands, LLC",3.0,NaN
1,Mar 3 2023 9:09AM,258610,19,"Graystone, LLC",1.0,NaN
2,Mar 3 2023 9:08AM,258609,19,"Graystone, LLC",1.0,NaN
3,Mar 3 2023 9:01AM,258607,10,Eywa Pharma Inc.,1.0,NaN
4,Mar 3 2023 9:01AM,258608,10,Emerginnova,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 3 2023 9:15AM,258611,10,"Digital Brands, LLC",3.0,0.0
1,Mar 3 2023 9:09AM,258610,19,"Graystone, LLC",1.0,0.0
2,Mar 3 2023 9:08AM,258609,19,"Graystone, LLC",1.0,0.0
3,Mar 3 2023 9:01AM,258607,10,Eywa Pharma Inc.,1.0,0.0
4,Mar 3 2023 9:01AM,258608,10,Emerginnova,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2844532,18.0,1.0
12,258599,1.0,0.0
19,2327029,55.0,9.0
22,258597,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2844532,18.0,1.0
1,12,258599,1.0,0.0
2,19,2327029,55.0,9.0
3,22,258597,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,18.0,1.0
1,12,1.0,0.0
2,19,55.0,9.0
3,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,18.0
1,12,Released,1.0
2,19,Released,55.0
3,22,Released,0.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,22
Status,,,,
Executing,1.0,0.0,9.0,1.0
Released,18.0,1.0,55.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,22
0,Executing,1.0,0.0,9.0,1.0
1,Released,18.0,1.0,55.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,22
0,Executing,1.0,0.0,9.0,1.0
1,Released,18.0,1.0,55.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()